# **Spark – Cálculo del índice TF-IDF**

In [ ]:
# Instalar Spark para Python
!pip install pyspark

import os

# Instalar Java SDK 8
!apt-get install -y openjdk-8-jdk -qq > /dev/null
!echo $(/usr/libexec/java_home -v 1.8)

# Set environment variable
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!echo 2 | update-alternatives --config java

In [ ]:
# Montar Google Drive en Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Definición de rutas
root_path = '/content/drive/MyDrive/Colab/A2/'
inputDir = root_path + "input/"
outputDir = root_path + "output/"
output_d = outputDir + 'output_d_spark'
output_idf_tf = outputDir + 'output_idf_tf_spark'

# Inicializa contexto
from pyspark import SparkContext
sc = SparkContext("local", "TF-IDF")

In [ ]:
# Cargar los datos de entrada como un RDD de pares clave-valor
input_data = sc.textFile(inputDir + "dataset.txt")

##Limpieza de los datos

In [ ]:
def only_recipes(line):
  lenght = len(line)

  if lenght >= 2:
    texto = line[1]
    return not texto.startswith('"Ingredientes:"')
  else:
    return False

In [ ]:
import re

# Función para limpiar el texto
def clean_text(text):
    caracteres_permitidos = r"A-Za-z0-9"
    words = re.split(r'\s+', text)
    cleaned_text = ""

    for word in words:
        if (word != ',' and word != ' '):
          cleaned_word = re.sub(f"[^({caracteres_permitidos})]", "", word)
          cleaned_text += cleaned_word + " "

    return cleaned_text

Función para gurdar los datos

In [ ]:
import shutil

def save_results(rdd, path):
  # Verificar si el directorio ya existe y eliminarlo si es necesario
  if os.path.exists(path):
      shutil.rmtree(path)

  # Guardar los resultados de TF
  rdd.saveAsTextFile(path)

In [62]:
# Limpieza de datos

# Mapea los archivos a lineas con el formato: Doc_id Texto
lines = input_data.map(lambda texto: texto.split("\t"))

# Filtra descartando las lineas las lineas que comiencen con "Ingredientes"
recipes = lines.filter(only_recipes)

# Mapea las lineas de Texto -> Texto limpio
cleaned_data = recipes.map(lambda x: (x[0], clean_text(x[1])))

def format_for_tf(t):
  id_doc = t[0]
  words = t[1].split()

  for w in words:
    yield ((id_doc, w), 1)

# Establecer clave (id_doc, termino) y valor 1
formated_data = cleaned_data.flatMap(format_for_tf)

# Unir múltiples partes con mismo id_doc en un solo documento
combined_data = cleaned_data.reduceByKey(lambda x, y: str(x) + " " + str(y))

In [60]:
print(formated_data.take(5))

[(('1', 'Para'), 1), (('1', 'hacer'), 1), (('1', 'la'), 1), (('1', 'receta'), 1), (('1', 'de'), 1)]


##Calculo de TF

In [100]:
# Calcular TF
tf_data = formated_data.reduceByKey(lambda x,y: x + y)

# Cantidad de palabras por documento (id_doc, cantidad de palabras)
words_per_document = combined_data.map(lambda t: (t[0], len(t[1].split())))

# Establece la clave como el id_doc y los valores como (termino, cantidad de veces que aparece en el documento)
prepare_to_join = tf_data.map(lambda t: (t[0][0], (t[0][1], int(t[1]))))

# Unifica la cantidad de veces que aparece cada termino por documento y la cantidad de terminos por documento
join_words_with_count = prepare_to_join.join(words_per_document)

# Calcula el TF por termino y documento
tf = join_words_with_count.map(lambda t: ((t[0], t[1][0][0]), float(t[1][0][1]/float(t[1][1]))))

print(tf.take(5))

[(('1', 'Para'), 0.004784688995215311), (('1', 'hacer'), 0.004784688995215311), (('1', 'la'), 0.028708133971291867), (('1', 'receta'), 0.004784688995215311), (('1', 'de'), 0.04784688995215311)]


##Calculo de D


In [101]:
# Calcular D
d_count = combined_data.count()
d_data = sc.parallelize([("D:", d_count)])

In [102]:
print(d_data.first())

('D:', 1908)


##Calculo de D(t)

In [91]:
# Para cada palabra genero un tupla de la forma <Palabra, 1>
words = tf.map(lambda x: (x[0][1], 1))

# Contar la cantidad de veces que aparece la clave: <Palabra>
word_frequency = words.reduceByKey(lambda x, y: x + y)

In [92]:
word_frequency.take(3)

[('en', 1892), ('y', 1908), ('calienta', 227)]

##Calculo de IDF


In [93]:
import math

total_documents = d_data.first()[1]

# Para cada termino se calcula log(#D / D(t)), donde #D es el total de Documentos y D(t) es la
# cantidad de veces que aparece el termino t en al menos uno de los documentos

idf_rdd = word_frequency.map(lambda x: (x[0], math.log(total_documents / x[1])))

In [94]:
print(idf_rdd.take(5))

[('en', 0.008421102396408345), ('y', 0.0), ('calienta', 2.1288608345268294), ('cucharadas', 1.8702310846695502), ('los', 0.087583295792751)]


##Calculo de TF-IDF


In [103]:
# Aplicar la función map al RDD: Dada las tuplas de la forma: Key, ('Palabra', Frecuencia_en_key-esimo_documento), retorna:
# 'Palabra(nueva_clave)', frecuencia
tf_format = tf.map(lambda t: (t[0][1], float(t[1])))

# Sumatorio de los TFs por termino
tf_sum = tf_format.reduceByKey(lambda x,y: x + y)

print(tf_sum.take(3))

[('en', 44.22505751027202), ('y', 76.2798744401357), ('calienta', 1.1176546891416206)]


In [97]:
# Unifica la sumatorio de TF por termino junto al IDF del termino
join = tf_sum.join(idf_rdd)

# Calcula el TF-IDF realizando el producto entre sumatoria(TF(t))*IDF(t)
res = join.mapValues(lambda t: t[0] * t[1])

In [104]:
# Guarda el resultado en el directorio "output_idf_tf"
save_results(res, output_idf_tf)

print(res.take(5))

[('y', 0.0), ('cucharadas', 2.915782900667218), ('los', 2.7359825398069217), ('hasta', 4.449240748528755), ('pimienta', 3.8295409058454504)]


#Diferencias entre Spark y MapReduce: Debugeo, Abstracción y Rendimiento


Al llevar a cabo el mismo trabajo con dos frameworks distintos, Spark y MapReduce, pudimos experimentar varias diferencias significativas. Al utilizar MapReduce, notamos que la capacidad de imprimir mensajes de depuración directamente en las funciones de Map o Reduce facilitó enormemente la identificación y resolución de problemas específicos en diferentes etapas del código.

Sin embargo, al migrar al uso de Spark, encontramos que no se podían incluir declaraciones de impresión directamente en las funciones de filtro y mapeo. Esta limitación dificultó la identificación rápida de problemas en etapas específicas del proceso de datos.

Además, enfrentamos la complejidad adicional de los errores en Spark, donde los mensajes de error a menudo mostraban una pila de errores de Java, lo que complicaba la identificación del origen exacto del problema, especialmente al trabajar con Python.

En cuanto al rendimiento y el flujo de trabajo, MapReduce requería la generación de archivos intermedios, lo que ralentizaba la ejecución debido a la escritura y lectura de estos archivos. En contraste, al utilizar Spark, experimentamos una ejecución más rápida y eficiente, ya que mantenía los datos en memoria, minimizando la necesidad de archivos intermedios. Esto no solo aceleró el proceso, sino que también redujo la complejidad del flujo de trabajo.

Además, Spark generalmente requiere menos líneas de código debido a sus abstracciones más elevadas, simplificando el desarrollo.